<a href="https://colab.research.google.com/github/DivyaAsija83/DL_p3_spam-quora/blob/main/DLProjectQuora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd

# Load dataset into a DataFrame
df = pd.read_csv('/content/train (1).csv')

# Display first 5 rows of dataset
df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [2]:
df.info

<bound method DataFrame.info of                           qid  \
0        00002165364db923c7e6   
1        000032939017120e6e44   
2        0000412ca6e4628ce2cf   
3        000042bf85aa498cd78e   
4        0000455dfa3e01eae3af   
...                       ...   
1306117  ffffcc4e2331aaf1e41e   
1306118  ffffd431801e5a2f4861   
1306119  ffffd48fb36b63db010c   
1306120  ffffec519fa37cf60c78   
1306121  ffffed09fedb5088744a   

                                             question_text  target  
0        How did Quebec nationalists see their province...       0  
1        Do you have an adopted dog, how would you enco...       0  
2        Why does velocity affect time? Does velocity a...       0  
3        How did Otto von Guericke used the Magdeburg h...       0  
4        Can I convert montra helicon D to a mountain b...       0  
...                                                    ...     ...  
1306117  What other technical skills do you need as a c...       0  
1306118  Does MS in ECE have good job prospects in USA ...       0  
1306119                          Is foam insulation toxic?       0  
1306120  How can one start a research project based on ...       0  
1306121  Who wins in a battle between a Wolverine and a...       0  

[1306122 rows x 3 columns]>

In [3]:
print(df.describe())

             target
count  1.306122e+06
mean   6.187018e-02
std    2.409197e-01
min    0.000000e+00
25%    0.000000e+00
50%    0.000000e+00
75%    0.000000e+00
max    1.000000e+00


In [4]:
# Check for missing values
missing_values = df.isnull().sum()

# Check class distribution
class_distribution = df['target'].value_counts(normalize=True)

missing_values, class_distribution

(qid              0
 question_text    0
 target           0
 dtype: int64,
 target
 0    0.93813
 1    0.06187
 Name: proportion, dtype: float64)

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Parameters
max_words = 20000
max_seq_length = 100

# Initialize tokenizer
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['question_text'])

# Convert texts to sequences
X = tokenizer.texts_to_sequences(df['question_text'])

# Padding sequences
X = pad_sequences(X, maxlen=max_seq_length)

# Target variable
y = df['target'].values


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(X_train.shape, X_val.shape)


(1044897, 100) (261225, 100)


In [7]:
import numpy as np

embeddings_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = vector

print(f'Total words in embeddings: {len(embeddings_index)}')


Total words in embeddings: 400000


In [8]:
word_index = tokenizer.word_index
num_words = min(max_words, len(word_index)) + 1
embedding_dim = 100

embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i < num_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector



In [9]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D

model = Sequential()
model.add(Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=max_seq_length, trainable=False))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │       2,000,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d (SpatialDropout1D) │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,000,100 (7.63 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,000,100 (7.63 MB)

In [10]:
history = model.fit(
    X_train, y_train,
    epochs=3,             # adjust epochs as needed
    batch_size=128,
    validation_data=(X_val, y_val),
    verbose=1
)


Epoch 1/3
8164/8164 ━━━━━━━━━━━━━━━━━━━━ 2359s 288ms/step - accuracy: 0.9454 - loss: 0.1508 - val_accuracy: 0.9538 - val_loss: 0.1181
Epoch 2/3
8164/8164 ━━━━━━━━━━━━━━━━━━━━ 2369s 290ms/step - accuracy: 0.9524 - loss: 0.1232 - val_accuracy: 0.9560 - val_loss: 0.1129
Epoch 3/3
8164/8164 ━━━━━━━━━━━━━━━━━━━━ 2411s 289ms/step - accuracy: 0.9542 - loss: 0.1174 - val_accuracy: 0.9569 - val_loss: 0.1106


In [11]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = (model.predict(X_val) > 0.5).astype("int32")

print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))


8164/8164 ━━━━━━━━━━━━━━━━━━━━ 471s 58ms/step
              precision    recall  f1-score   support

           0       0.97      0.98      0.98    245063
           1       0.70      0.53      0.60     16162

    accuracy                           0.96    261225
   macro avg       0.83      0.76      0.79    261225
weighted avg       0.95      0.96      0.95    261225

[[241349   3714]
 [  7546   8616]]
